In [1]:
#@title
import os
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# coding=utf-8
from collections import defaultdict
import pandas as pd
from gensim.models import Word2Vec

In [3]:
############################ 加载数据 ############################
df_all = pd.read_csv("drive/My Drive/sougou/data/all_data.csv",encoding="UTF-8",index_col=0)
df_all['Age'] = df_all['Age'] - 1
df_all['Gender'] = df_all['Gender'] - 1
df_all['Education'] = df_all['Education'] - 1
df_all.head()

,ID,Age,Gender,Education,Query_List,Query_List1,jieba,jieba2
0,22DD920316420BE2DF8D6EE651BA174B,0.0,0.0,3.0,柔和双沟\t女生\t中财网首页 财经\thttp://pan.baidu.com/s/1pl...,柔和双沟\t女生\t中财网首页 财经\t网址\t周公解梦大全查询2345\t曹云金再讽郭德纲...,柔和 双沟 女生 中财网 首页 财经 网址 周公 解梦 大全 查询 2345 曹云金 再讽 ...,柔和 双沟 女生 中财网 首页 财经 网址 周公 解梦 大全 查询 2345 曹云金 再...
1,43CC3AF5A8D6430A3B572337A889AFE4,1.0,0.0,2.0,"广州厨宝烤箱\t世情薄,人情恶,雨送黄昏花易落,晓风干,泪痕\t厦门酒店用品批发市场\t我只...","广州厨宝烤箱\t世情薄,人情恶,雨送黄昏花易落,晓风干,泪痕\t厦门酒店用品批发市场\t我只...",广州 厨宝 烤箱 世情 人情 雨送 黄昏 花易落 风干 泪痕 厦门 酒店用品 批发市场 不想...,"广州 厨宝 烤箱 世情 薄 , 人情 恶 , 雨送 黄昏 花易落 , 晓 风干 , 泪痕 厦..."
2,6931EFC26D229CCFCEA125D3F3C21E57,3.0,1.0,2.0,最受欢迎狗狗排行榜\t舶怎么读\t场景描 写范例\t三维绘图软件\t枣和酸奶能一起吃吗\t好...,最受欢迎狗狗排行榜\t舶怎么读\t场景描 写范例\t三维绘图软件\t枣和酸奶能一起吃吗\t好...,受欢迎 狗狗 排行榜 场景 范例 三维 绘图 软件 酸奶 壮观 衣服 网站 动漫 绘图 软件...,最 受欢迎 狗狗 排行榜 舶 怎么 读 场景 描 写 范例 三维 绘图 软件 枣 和 酸...
3,E780470C3BB0D340334BD08CDCC3C71A,1.0,1.0,3.0,干槽症能自愈吗\t太太万岁叶舒心去没去美国\t干槽症\t右眼皮下面一直跳是怎么回事\t麦当劳...,干槽症能自愈吗\t太太万岁叶舒心去没去美国\t干槽症\t右眼皮下面一直跳是怎么回事\t麦当劳...,干槽症 自愈 太太 万岁 舒心 美国 干槽症 眼皮 怎么回事 麦当劳 旋风 勺子 吉林市 鹿...,干槽症 能 自愈 吗 太太 万岁 叶 舒心 去 没 去 美国 干槽症 右 眼皮 下面 一直 ...
4,0D58894CD481D4B3039000661665C0DB,2.0,1.0,2.0,貔貅饲育指南\t韩姨\t贵阳学古琴\t洛基\t风速狗\t综英美]店主不好吃\t>重生之被拐儿...,貔貅饲育指南\t韩姨\t贵阳学古琴\t洛基\t风速狗\t综英美]店主不好吃\t>重生之被拐儿...,貔貅 饲育 指南 韩姨 贵阳 古琴 洛基 风速 英美 店主 好吃 重生 儿童 寻家记 锦里 ...,貔貅 饲育 指南 韩姨 贵阳 学 古琴 洛基 风速 狗 综 英美 ] 店主 不 好吃 > 重...


In [4]:
df_all.shape

(188492, 8)

In [0]:
import time
def log(stri):
  now = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())
  print(str(now) + ' ' + str(stri))

In [7]:
############################ w2v ############################
documents = df_all['jieba2'].values
log('documents number %d' % len(documents))

texts = [[word for word in document.split(' ')] for document in documents]
frequency = defaultdict(int)
for text in texts:
  for token in text:
    frequency[token] += 1
texts = [[token for token in text if frequency[token] >= 5] for text in texts]

log('Train Model...')
w2v = Word2Vec(texts, size=300, window=5, iter=15, workers=12, seed=2020)
w2v.save("drive/My Drive/sougou/model/w2v_21w.model")
log('Save done!')

2020-04-12 09:08:21 documents number 188492
2020-04-12 09:09:46 Train Model...


/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:410: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


2020-04-12 09:46:04 Save done!


In [9]:
import numpy as np
############################ 加载模型 ############################
model = Word2Vec.load("drive/My Drive/sougou/model/w2v_21w.model")

############################ w2v ############################
log('Start get w2v feat..')
w2v_feat = np.zeros((len(texts),300))
w2v_feat_avg = np.zeros((len(texts), 300))
i = 0
for line in texts:
    num = 0
    for word in line:
        num += 1
        vec = model[word]
        w2v_feat[i, :] += vec
    w2v_feat_avg[i, :] = w2v_feat[i, :] / num
    i += 1
    if i % 1000 == 0:
        log(i)

df_w2v = pd.DataFrame(w2v_feat)
df_w2v.columns = ['w2v_' + str(i) for i in df_w2v.columns]
df_w2v.to_csv("drive/My Drive/sougou/data/w2v.csv", encoding='utf8', index=None)
df_w2v_avg = pd.DataFrame(w2v_feat_avg)
df_w2v_avg.columns = ['w2v_avg_' + str(i) for i in df_w2v_avg.columns]
df_w2v_avg.to_csv("drive/My Drive/sougou/data/w2v_avg.csv", encoding='utf8', index=None)

log('Save w2v and w2v_avg feat done!')

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:410: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


2020-04-12 10:13:39 Start get w2v feat..


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


2020-04-12 10:13:44 1000
2020-04-12 10:13:47 2000
2020-04-12 10:13:51 3000
2020-04-12 10:13:55 4000
2020-04-12 10:13:58 5000
2020-04-12 10:14:02 6000
2020-04-12 10:14:06 7000
2020-04-12 10:14:09 8000
2020-04-12 10:14:13 9000
2020-04-12 10:14:16 10000
2020-04-12 10:14:20 11000
2020-04-12 10:14:24 12000
2020-04-12 10:14:27 13000
2020-04-12 10:14:31 14000
2020-04-12 10:14:35 15000
2020-04-12 10:14:38 16000
2020-04-12 10:14:42 17000
2020-04-12 10:14:46 18000
2020-04-12 10:14:49 19000
2020-04-12 10:14:53 20000
2020-04-12 10:14:57 21000
2020-04-12 10:15:00 22000
2020-04-12 10:15:04 23000
2020-04-12 10:15:08 24000
2020-04-12 10:15:11 25000
2020-04-12 10:15:15 26000
2020-04-12 10:15:19 27000
2020-04-12 10:15:22 28000
2020-04-12 10:15:26 29000
2020-04-12 10:15:29 30000
2020-04-12 10:15:33 31000
2020-04-12 10:15:37 32000
2020-04-12 10:15:40 33000
2020-04-12 10:15:44 34000
2020-04-12 10:15:48 35000
2020-04-12 10:15:51 36000
2020-04-12 10:15:55 37000
2020-04-12 10:15:58 38000
2020-04-12 10:16:02 3

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: RuntimeWarning: invalid value encountered in true_divide
  app.launch_new_instance()


2020-04-12 10:20:20 109000
2020-04-12 10:20:24 110000
2020-04-12 10:20:27 111000
2020-04-12 10:20:31 112000
2020-04-12 10:20:35 113000
2020-04-12 10:20:39 114000
2020-04-12 10:20:43 115000
2020-04-12 10:20:46 116000
2020-04-12 10:20:50 117000
2020-04-12 10:20:54 118000
2020-04-12 10:20:58 119000
2020-04-12 10:21:02 120000
2020-04-12 10:21:05 121000
2020-04-12 10:21:09 122000
2020-04-12 10:21:13 123000
2020-04-12 10:21:17 124000
2020-04-12 10:21:21 125000
2020-04-12 10:21:25 126000
2020-04-12 10:21:29 127000
2020-04-12 10:21:33 128000
2020-04-12 10:21:36 129000
2020-04-12 10:21:40 130000
2020-04-12 10:21:44 131000
2020-04-12 10:21:48 132000
2020-04-12 10:21:51 133000
2020-04-12 10:21:55 134000
2020-04-12 10:21:59 135000
2020-04-12 10:22:03 136000
2020-04-12 10:22:06 137000
2020-04-12 10:22:10 138000
2020-04-12 10:22:14 139000
2020-04-12 10:22:18 140000
2020-04-12 10:22:21 141000
2020-04-12 10:22:25 142000
2020-04-12 10:22:29 143000
2020-04-12 10:22:33 144000
2020-04-12 10:22:37 145000
2